De code voor het SVM gedeelte van de opdracht

In [ ]:
from worcliver.load_data import load_data
#from ecg.load_data import load_data

data = load_data("TrainData.csv")
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

ModuleNotFoundError: No module named 'load_data'

In [ ]:
# Create the RFE object and compute a cross-validated score.
svc = svm.SVC(kernel="linear")

# classifications
rfecv = feature_selection.RFECV(
    estimator=svc, step=1,
    cv=model_selection.StratifiedKFold(4),
    scoring='roc_auc')
rfecv.fit(X2, y2)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.cv_results_["mean_test_score"]) + 1), rfecv.cv_results_["mean_test_score"])
plt.show()

print("Optimal number of features:", rfecv.n_features_)
print("Selected features:", rfecv.support_)
print("Feature ranking:", rfecv.ranking_)